# Create a Target Forecast

* **About 50 mins may be elapsed**

In [1]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
# Recover variables stored by other notebooks
%store -r

In [3]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create Prophet and DeepAR+ Forecast

In [4]:
# Prophet
prophet_forecastName = project+'_prophet_algo_forecast' + target_suffix + suffix
prophet_create_forecast_response=forecast.create_forecast(
    ForecastName=prophet_forecastName,
    PredictorArn=target_prophet_predictorArn)
target_prophet_forecast_arn = prophet_create_forecast_response['ForecastArn']

In [5]:
forecast.describe_forecast(ForecastArn = target_prophet_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/StoreItemDemand_prophet_algo_forecast_target30517',
 'ForecastName': 'StoreItemDemand_prophet_algo_forecast_target30517',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/StoreItemDemand_prophet_algo_1_target30517',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/StoreItemDemandDSG30517',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 4, 10, 45, 34, 228000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 4, 10, 45, 34, 228000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c0949579-860b-40bd-86d9-6b8be86f5d24',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 10:45:35 GMT',
   'x-amzn-requestid': 'c0949579-860b-40bd-86d9-6b8be86f5d24',
   'content-length': '525',
   'connection': 'keep-alive'},
  'RetryAttempts'

In [6]:
# DeepAR+
deeparp_forecastName = project+'_deeparp_algo_forecast' + target_suffix + suffix
deeparp_create_forecast_response=forecast.create_forecast(
    ForecastName=deeparp_forecastName,
    PredictorArn=target_deepar_predictorArn)
target_deeparp_forecast_arn = deeparp_create_forecast_response['ForecastArn']

In [7]:
forecast.describe_forecast(ForecastArn = target_deeparp_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/StoreItemDemand_deeparp_algo_forecast_target30517',
 'ForecastName': 'StoreItemDemand_deeparp_algo_forecast_target30517',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:us-east-2:057716757052:predictor/StoreItemDemand_deeparp_algo_130517',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/StoreItemDemandDSG30517',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 4, 10, 45, 36, 585000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 4, 10, 45, 36, 585000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'fbf53f23-0972-4b31-bd23-8dde51a028e3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 10:45:37 GMT',
   'x-amzn-requestid': 'fbf53f23-0972-4b31-bd23-8dde51a028e3',
   'content-length': '518',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [12]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast(ForecastArn= target_prophet_forecast_arn)['Status']
    createDeeparpStatus = forecast.describe_forecast(ForecastArn= target_deeparp_forecast_arn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 13.7 ms, sys: 366 µs, total: 14.1 ms
Wall time: 114 ms


## Upload forecast results to S3

In [13]:
target_prophet_path = "s3://" + bucket_name + "/" + bucket_folder + "/prophet_" + target_suffix + suffix + "/" 
target_prophet_job_name = "ProphetExport1" + target_suffix + suffix
create_forecast_export_job_prophet_response = forecast.create_forecast_export_job(
    ForecastExportJobName = target_prophet_job_name,
    ForecastArn = target_prophet_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": target_prophet_path,
            "RoleArn": role_arn
        }
    })


ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateForecastExportJob operation: A forecast export already exists with the arn: arn:aws:forecast:us-east-2:057716757052:forecast-export-job/StoreItemDemand_prophet_algo_forecast_target30517/ProphetExport1_target30517

In [14]:
TargetForecastProphetExportJobArn = create_forecast_export_job_prophet_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = TargetForecastProphetExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:us-east-2:057716757052:forecast-export-job/StoreItemDemand_prophet_algo_forecast_target30517/ProphetExport1_target30517',
 'ForecastExportJobName': 'ProphetExport1_target30517',
 'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/StoreItemDemand_prophet_algo_forecast_target30517',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-us-east-2-057716757052/StoreItemDemand/prophet__target30517',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRolePOC30517'}},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 4, 11, 19, 44, 83000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 4, 11, 21, 31, 97000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'f3218234-4c6e-4022-843c-58c111af2319',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 11:21:42 GMT',
   'x-amzn-requestid': 'f3218234-4c6e-4022-843c-58c111af2319',
   '

In [18]:
target_deeparp_path = "s3://" + bucket_name + "/" +  bucket_folder +  "/deeparp_" + target_suffix + suffix + "/" 
target_deeparp_job_name = "DeepARPExport1" + target_suffix + suffix
create_forecast_export_job_deepar_response = forecast.create_forecast_export_job(
    ForecastExportJobName = target_deeparp_job_name,
    ForecastArn = target_deeparp_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": target_deeparp_path,
            "RoleArn": role_arn
        }
    })

In [19]:
TargetForecastDeeparExportJobArn = create_forecast_export_job_deepar_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = TargetForecastDeeparExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:us-east-2:057716757052:forecast-export-job/StoreItemDemand_deeparp_algo_forecast_target30517/DeepARPExport1_target30517',
 'ForecastExportJobName': 'DeepARPExport1_target30517',
 'ForecastArn': 'arn:aws:forecast:us-east-2:057716757052:forecast/StoreItemDemand_deeparp_algo_forecast_target30517',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-us-east-2-057716757052/StoreItemDemand/deeparp__target30517',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRolePOC30517'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 4, 4, 11, 22, 48, 321000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 4, 11, 22, 48, 321000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '0b33f309-72a5-48bb-a1fa-e9b730bf4499',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 11:22:49 GMT',
   'x-amzn-requestid': '0b33f309-72a5-48bb-a1fa-e9b730bf4

This exporting process is another one of those items that will take several minutes to complete. Just poll for progress in the console. From the earlier page where you saw the status turn `Active` for a Forecast, click it and you can see the progress of the export.

In [23]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= TargetForecastProphetExportJobArn)['Status']
    createDeeparpStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= TargetForecastDeeparExportJobArn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 4.73 ms, sys: 29 µs, total: 4.76 ms
Wall time: 70.5 ms


In [24]:
%store target_prophet_forecast_arn
%store target_deeparp_forecast_arn
%store TargetForecastProphetExportJobArn
%store TargetForecastDeeparExportJobArn



Stored 'target_prophet_forecast_arn' (str)
Stored 'target_deeparp_forecast_arn' (str)
Stored 'TargetForecastProphetExportJobArn' (str)
Stored 'TargetForecastDeeparExportJobArn' (str)
